In [1]:
import cv2 as cv

In [2]:
import numpy as np

In [3]:
import open3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
from tqdm import tqdm

In [5]:
from PIL import Image

In [29]:
def create_point_fill(points, resolution=0.1, accumulation=None):
    """
    Create and save an image of points within a rectangular outline at specified resolution.
    
    Parameters:
    points: List of tuples [(x1,y1), (x2,y2), ...]
    resolution: Float, grid cell size
    save_path: String, path to save the output image
    """
    # Convert points to numpy array
    points = np.array(points)
    
    # Find min and max values for x and y
    x_min, x_max = points[:, 0].min(), points[:, 0].max()
    y_min, y_max = points[:, 1].min(), points[:, 1].max()
    
    # Create grid based on resolution
    x_grid = np.arange(x_min, x_max + resolution, resolution)
    y_grid = np.arange(y_min, y_max + resolution, resolution)
    grid_x, grid_y = np.meshgrid(x_grid, y_grid)
    
    # Create empty grid
    grid = np.zeros((len(y_grid), len(x_grid)))
    
    # Fill in the points
    for point in tqdm(points):
        # Find closest grid indices
        x_idx = int((point[0] - x_min) / resolution)
        y_idx = int((point[1] - y_min) / resolution)
        
        # Ensure indices are within bounds
        x_idx = min(x_idx, len(x_grid) - 1)
        y_idx = min(y_idx, len(y_grid) - 1)
        
        # Mark point on grid
        if not accumulation:
            grid[y_idx, x_idx] = 1
        elif accumulation=="height":
            grid[y_idx, x_idx] += point[2]
        else:
            grid[y_idx, x_idx] += 1
    return grid


In [35]:
def extract_points_from_image(grid, resolution=0.1, x_min=0, y_min=0):
    """
    Extract points from a heightmap image based on non-zero pixel locations.
    
    Parameters:
    grid: 2D numpy array with height values
    resolution: Float, grid cell size
    x_min: Float, minimum x coordinate in the original coordinate system
    y_min: Float, minimum y coordinate in the original coordinate system
    
    Returns:
    points: List of tuples [(x1, y1, z1), (x2, y2, z2), ...] in the original coordinate system
    """
    import numpy as np
    
    # Find non-zero pixel locations
    y_indices, x_indices = np.nonzero(grid)
    
    # Convert indices back to original coordinate system
    x_coords = x_min + (x_indices * resolution)
    y_coords = y_min + (y_indices * resolution)
    z_coords = grid[y_indices, x_indices]  # Get height values
    
    # Combine into points list
    points = list(zip(x_coords, y_coords, z_coords))
    
    return points

In [8]:
pcd = open3d.io.read_point_cloud("../data/driveable_full.pcd")

In [15]:
downpcd = pcd.voxel_down_sample(voxel_size=0.1)

In [9]:
downpoints = np.asarray(pcd.points)

In [10]:
grid = create_point_fill(downpoints[:, :2], resolution=0.05)

100%|██████████████████████████| 20867679/20867679 [00:16<00:00, 1292129.92it/s]


In [80]:
hgrid = create_point_fill(downpoints, resolution=0.05, accumulation="height")

100%|██████████████████████████| 20867679/20867679 [00:19<00:00, 1073875.49it/s]


In [30]:
fgrid = create_point_fill(downpoints, resolution=0.05, accumulation="count")

100%|██████████████████████████| 20867679/20867679 [00:18<00:00, 1113536.31it/s]


In [49]:
(fgrid!=0).sum()

np.int64(8242855)

In [82]:
ground_height = np.divide(hgrid, fgrid+1e-7)

In [84]:
ground_height.min()

np.float64(0.0)

In [85]:
np.save("../data/ground_height_map.npy", ground_height)

In [11]:
grid = np.astype(grid*255, np.uint8)

In [12]:
cv.imwrite("driveable_img.jpeg", grid.squeeze())

True

In [13]:
img = cv.imread("driveable_img.jpeg", cv.CV_8U)

In [14]:
erosion_size=2
erosion_shape=cv.MORPH_RECT
element = cv.getStructuringElement(erosion_shape, (2 * erosion_size + 1, 2 * erosion_size + 1),
                                       (erosion_size, erosion_size))
eimg = cv.dilate(img, element)
eimg = cv.dilate(eimg, element)
eimg = cv.dilate(eimg, element)
eimg = cv.dilate(eimg, element)
eimg = cv.dilate(eimg, element)

In [15]:
dst = cv.Canny(eimg, 50, 200, None, 3)

In [16]:
cv.imwrite("driveable_img_canny.png", dst)

True

In [17]:
img = Image.open("driveable_img_canny.png")

In [18]:
iarr = np.asarray(img)

In [34]:
x_min=downpoints[:, 0].min()
y_min=downpoints[:, 1].min()

In [21]:
points = extract_points_from_image(iarr, 0.05, x_min, y_min)

In [22]:
points = np.array(points)

In [23]:
dpoints = np.hstack([points, np.zeros((points.shape[0], 1))])

In [24]:
opoints = open3d.utility.Vector3dVector(dpoints)

In [25]:
bpcd = open3d.geometry.PointCloud(opoints)

In [26]:
open3d.io.write_point_cloud("../data/driveable_boundary_full.pcd", bpcd)

True

In [86]:
points = extract_points_from_image(ground_height, 0.05, x_min, y_min)

In [87]:
points = np.array(points)

In [88]:
points

array([[  51.41673889, -183.14494324,   27.33546365],
       [  51.36673889, -183.09494324,   27.33309663],
       [  51.36673889, -183.04494324,   27.33601869],
       ...,
       [  87.56673889,  109.15505676,   36.62374512],
       [  88.31673889,  109.15505676,   36.6101152 ],
       [  87.61673889,  109.20505676,   36.62328354]])

In [89]:
dpoints = points

In [90]:
dpoints

array([[  51.41673889, -183.14494324,   27.33546365],
       [  51.36673889, -183.09494324,   27.33309663],
       [  51.36673889, -183.04494324,   27.33601869],
       ...,
       [  87.56673889,  109.15505676,   36.62374512],
       [  88.31673889,  109.15505676,   36.6101152 ],
       [  87.61673889,  109.20505676,   36.62328354]])

In [91]:
opoints = open3d.utility.Vector3dVector(dpoints)

In [92]:
bpcd = open3d.geometry.PointCloud(opoints)

In [93]:
open3d.io.write_point_cloud("../data/driveable_ground_height.pcd", bpcd)

True

In [94]:
sum(dpoints[:, 2]==0.9999999000000099)

np.int64(0)

In [95]:
dpoints[:, 2].max()

np.float64(58.236108678342255)

In [96]:
dpoints[:, 2].min()

np.float64(23.25882869828705)